In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_pickle('data.pkl')
data = data[data.date_block_num > 11]
data_og = data.copy()

In [3]:
drop_features = ['date_cat-medium-type_avg_item_cnt_lag_1', 'date_city_avg_item_cnt_lag_2', 'date_shop_avg_item_cnt_lag_6', 'days_in_month', 'delta_price_trend_with_lag_1', 'delta_price_trend_with_lag_4', 'delta_shop_revenue_this_month', 'delta_shop_revenue_this_month_lag_2', 'item_avg_price_during_month_lag_3']
data.drop(drop_features, axis=1, inplace=True)

In [4]:
X = None
Y = None
x = None

def set_X_Y_x(df):
    global X, Y, x
    X = df[df.date_block_num < 34].drop(['item_cnt_month'], axis=1)
    Y = df[df.date_block_num < 34]['item_cnt_month']
    x = df[df.date_block_num == 34].drop(['item_cnt_month'], axis=1)

set_X_Y_x(data)

In [5]:
from sklearn.metrics import mean_squared_error

def rmse(y_te, y_p):
    return np.sqrt(mean_squared_error(y_te, y_p))

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65)
lf = LinearRegression()
lf.fit(X_train, y_train)
prediction = lf.predict(X_test).clip(0,20)

og_rmse = rmse(y_test, prediction)

print(f"RMSE: {og_rmse}")

RMSE: 0.7968769946792156


In [9]:
from itertools import product

train = pd.read_csv('data/technical/sales_train.csv')
test = pd.read_csv('data/technical/test.csv')

matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train[train.date_block_num==i]
    # product makes a cartesian product of all the columns. the labels
    # will be added again later on
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))

matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)

train['revenue'] = train['item_price'] *  train['item_cnt_day']


group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                            .fillna(0)
                            .clip(0,20) # NB clip target here
                            .astype(np.float16))


test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)


matrix = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True) # 34 month

X = matrix[matrix.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y = matrix[matrix.date_block_num < 33]['item_cnt_month']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65)

LR = LinearRegression()
LR.fit(X_train, y_train)
prediction = LR.predict(X_test)

print(f"RMSE: {rmse(y_test, prediction)}")

RMSE: 1.2232152290327236


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65)

LR = LinearRegression()
LR.fit(X_train, y_train)
prediction = LR.predict(X_test).clip(0,20)

print(f"RMSE: {rmse(y_test, prediction)}")

RMSE: 1.2248942299923087
